## <center>Natural Language Processing (Tugas 2)</center>
<strong> NIM  : 1301178418 </strong><br>
<strong> Nama : Bastomy </strong> <br>
<strong> Kelas : ICM-39-GAB </strong>

#### Deskripsi Tugas

Deskripsi
Buatlah program yang mengimplementasikan dua pendekatan POS-tagging. Sebagai input adalah
kalimat Bahasa Indonesia and output adalah kalimat setelah setiap katanya diberi tag yang tepat.
Kedua pendekatan yang harus diimplementasikan adalah:
1. Baseline (menggunakan POS Tag yang paling sering muncul)
2. HMM.

### Import library

In [2]:
import pandas as pd
import re
import operator

<strong>pandas</strong> =  untuk mempermudah membuat array menjadi data tabel <br>
<strong>re</strong> = regular expression untuk membagi string menjadi array berdasarkan tab (\t) <br>
<strong>operator</strong> = untuk mengambil item

### Fungsi untuk membaca data training dan testing menjadi array

In [3]:
def read_data(train):
    dt_train = {'kata':[], 'tag':[]}
    for line in open(train):
        if not line.startswith('<kalimat') and not line.startswith('</kalimat'):
            baris = re.split(r'\t+', line.rstrip('\t'))
            dt_train['kata'].append(baris[0].lower())
            dt_train['tag'].append(baris[1].replace('\n',''))
    data = pd.DataFrame(dt_train)        
    return data

read_data adalah fungsi untuk membaca data .txt menjadi sebuah array <br>
terdapat dua kolom array yaitu kata dan tag  <br>
setiap baris data akan di looping dan di cek jika tidak berawalan <kalimat dan </kalimat maka akan lanjut ke tahap berikutnya yaitu membagi setiap baris menjadi array berdasarkan \t (Tab) <br>
kemudian akan di simpan ke dt_train dan di set menjadi Dataframe untuk di return

### Fungsi untuk menghitung jumlah tag berdasarkan kata

In [4]:
def post_tag_frequency(kata, data_training):
    hitungTag = data_training.loc[data_training['kata'] == kata]['tag'].value_counts()
    if(len(hitungTag) == 0):
        return 'NN','0'
    tagTerbaik = max(hitungTag.items(),key=operator.itemgetter(1))
    return tagTerbaik[0],tagTerbaik[1]

Menghitung jumlah tag berdasarkan kata  <br>
variabel hitungTag akan menyimpan jumlah tag berdasarkan kata  <br>
jika tag bernilai 0 maka akan di terun tag NN dengan jumlah 0  (untuk menghandle kata yg tidak terdaftar)<br>
kemudian akan dicari max dari tag yg tersedia  <br>
fungsi akan mengembalikan Tag dan jumlah tag nya <br>
contoh (PRP   44) <br>

### Fungsi prediksi tag BASELINE

In [110]:
def baseline(data_testing,data_training):
    index = 0 
    tag_correct = 0
    result = {'kata':[], 'prediksi':[], 'tag':[]}
    akurasi = 0
    for i in data_testing['kata']:
        tag,count = post_tag_frequency(i, data_training)
        if(tag == data_testing['tag'][index]):
            tag_correct+=1
        result['kata'].append(i)
        result['prediksi'].append(tag)
        result['tag'].append(data_testing['tag'][index])
        index+=1
    data = pd.DataFrame(result)
    akurasi = (tag_correct/index)*100
    return data,akurasi

Baseline akan memprediksi Tag berdasarkan kemunculan terbanyak <br>
cara kerja fungsi diatas adalah melooping semua kata beserta Tag <br>
kemudian kata tersebut akan diprediksi Tag nya berdasarkan data training <br>
post_tag_frequency akan mengembalikan tag beserta jumlahnya kemunculannya, tag dari post_tag_frequency akan dijadikan prediksi <br>
akurasi dihitung dari jumlah tag yg benar dibagi dengan total seluruh kata

### Read Data

In [111]:
dt_training = read_data('corpus/training.txt')
dt_testing = read_data('corpus/testing.txt')

simpan data training dan testing kedalam variabel

### NO 1. Baseline (menggunakan POS Tag yang paling sering muncul)

In [112]:
data, akurasi = baseline(dt_testing,dt_training)

Data testing dan prediksi TAG

In [113]:
data

,kata,prediksi,tag
0,perusahaan,NN,NN
1,ritel,NN,NN
2,pt,NNP,NNP
3,matahari,NNP,NNP
4,putra,NN,NNP
5,prima,NNP,NNP
6,tbk,NNP,NNP
7,mppa,NN,NNP
8,dalam,IN,IN
9,tiga,CD,CD


Akurasi Tag (persen)

In [16]:
akurasi

87.97595190380761

### No 2 HMM

In [71]:
def read_file_init_table(fname):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    idx_line = 0
    is_first_word = 0
    
    while idx_line < len(content):
        prev_tag = '<start>'
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                if content_part[1] in tag_count:
                    tag_count[content_part[1]] += 1
                else:
                    tag_count[content_part[1]] = 1
                    
                current_word_tag = content_part[0]+','+content_part[1]
                if current_word_tag in word_tag:
                    word_tag[current_word_tag] += 1
                else:    
                    word_tag[current_word_tag] = 1
                    
                if is_first_word == 1:
                    current_tag_trans = '<start>,'+content_part[1]
                    is_first_word = 0
                else:
                    current_tag_trans = prev_tag+','+content_part[1]
                    
                if current_tag_trans in tag_trans:
                    tag_trans[current_tag_trans] += 1
                else:
                    tag_trans[current_tag_trans] = 1                    
                prev_tag = content_part[1]   
                
            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        idx_line = idx_line+1       
    return tag_count, word_tag, tag_trans

In [72]:
tag_count, word_tag, tag_trans = read_file_init_table('corpus/training.txt')

In [73]:
tag_count

{'<start>': 1030,
 'NN': 6464,
 'SC': 1158,
 'VB': 2852,
 'NNP': 3868,
 'JJ': 1076,
 'RB': 562,
 'IN': 2263,
 'Z': 2752,
 'CD': 2372,
 'CC': 811,
 'PR': 508,
 'PRP': 706,
 'MD': 534,
 'FW': 421,
 'NEG': 127,
 'DT': 20,
 'NND': 149,
 'SYM': 331,
 'RP': 18,
 'WH': 8,
 'OD': 98,
 'X': 15,
 'UH': 4}

In [74]:
word_tag

{'Kera,NN': 1,
 'untuk,SC': 126,
 'amankan,VB': 1,
 'pesta olahraga,NN': 1,
 'Pemerintah,NNP': 2,
 'kota,NNP': 3,
 'Delhi,NNP': 5,
 'mengerahkan,VB': 1,
 'monyet,NN': 12,
 'mengusir,VB': 4,
 'monyet-monyet,NN': 2,
 'lain,JJ': 29,
 'yang,SC': 538,
 'berbadan,VB': 2,
 'lebih,RB': 80,
 'kecil,JJ': 14,
 'dari,IN': 251,
 'arena,NN': 2,
 'Pesta Olahraga,NNP': 1,
 'Persemakmuran,NNP': 1,
 '.,Z': 1012,
 'Beberapa,CD': 1,
 'laporan,NN': 15,
 'menyebutkan,VB': 10,
 'setidaknya,RB': 2,
 '10,CD': 21,
 'ditempatkan,VB': 2,
 'di,IN': 360,
 'luar,NN': 5,
 'lomba,NN': 1,
 'dan,CC': 481,
 'pertandingan,NN': 1,
 'ibukota,NNP': 2,
 'India,NNP': 6,
 'Pemkot,NNP': 1,
 'memiliki,VB': 20,
 '28,CD': 6,
 'berencana,VB': 9,
 'mendatangkan,VB': 2,
 'sejenis,NN': 1,
 'negara bagian,NNP': 1,
 'Rajasthan,NNP': 1,
 'Jumlah,NN': 5,
 'mencapai,VB': 97,
 'ribuan,CD': 1,
 '",",Z': 1393,
 'sebagian besar,NN': 3,
 'berada,VB': 14,
 'kantor-kantor,NN': 1,
 'pemerintah,NN': 52,
 'hewan,NN': 1,
 'ini,PR': 199,
 'dianggap,VB'

In [75]:
tag_trans

{'<start>,NN': 321,
 'NN,SC': 437,
 'SC,VB': 496,
 'VB,NN': 938,
 '<start>,NNP': 169,
 'NNP,NNP': 1959,
 'NNP,VB': 272,
 'NN,JJ': 494,
 'JJ,SC': 90,
 'VB,RB': 65,
 'RB,JJ': 137,
 'JJ,IN': 154,
 'IN,NN': 1280,
 'NN,NNP': 509,
 'NNP,Z': 743,
 '<start>,CD': 26,
 'CD,NN': 432,
 'NN,VB': 296,
 'RB,CD': 40,
 'VB,IN': 479,
 'NN,NN': 2103,
 'NN,CC': 238,
 'CC,NN': 334,
 'NN,IN': 533,
 'IN,NNP': 440,
 'VB,CD': 253,
 'CC,VB': 86,
 'VB,VB': 140,
 'CD,Z': 434,
 'Z,NN': 386,
 'NN,PR': 324,
 'PR,VB': 140,
 'JJ,Z': 204,
 'VB,JJ': 80,
 'Z,VB': 331,
 'Z,CC': 127,
 'JJ,CC': 66,
 'VB,PRP': 174,
 'PRP,MD': 45,
 'MD,VB': 408,
 'SC,NN': 206,
 'JJ,JJ': 29,
 'JJ,VB': 74,
 'VB,Z': 189,
 'CC,FW': 19,
 'FW,VB': 22,
 'SC,RB': 74,
 'RB,VB': 171,
 'NN,Z': 504,
 'JJ,MD': 20,
 'IN,CD': 239,
 '<start>,Z': 26,
 'PR,SC': 17,
 'Z,Z': 54,
 'VB,NNP': 158,
 'NNP,IN': 261,
 '<start>,IN': 182,
 'CC,RB': 37,
 '<start>,NEG': 1,
 'NEG,JJ': 19,
 'JJ,PR': 22,
 'PR,NN': 31,
 'PR,NEG': 15,
 'NEG,MD': 22,
 'VB,SC': 137,
 'PRP,JJ': 11

In [76]:
def create_trans_prob_table(tag_trans, tag_count):
#     print(tag_trans)
    trans_prob = {}
    for tag1 in tag_count.keys():
        for tag2 in tag_count.keys():
            #print('tag1 = ')
            #print(tag1)
            trans_idx = tag1+','+tag2
            #print('trans_idx = ')
            #print(trans_idx)
            if trans_idx in tag_trans:
                #print(trans_idx)
                trans_prob[trans_idx] = tag_trans[trans_idx]/tag_count[tag1]
    return trans_prob

In [77]:
trans_prob = create_trans_prob_table(tag_trans, tag_count)

In [78]:
trans_prob

{'<start>,NN': 0.3116504854368932,
 '<start>,SC': 0.062135922330097085,
 '<start>,VB': 0.02912621359223301,
 '<start>,NNP': 0.16407766990291262,
 '<start>,JJ': 0.003883495145631068,
 '<start>,RB': 0.01262135922330097,
 '<start>,IN': 0.1766990291262136,
 '<start>,Z': 0.02524271844660194,
 '<start>,CD': 0.02524271844660194,
 '<start>,CC': 0.0854368932038835,
 '<start>,PR': 0.01650485436893204,
 '<start>,PRP': 0.06990291262135923,
 '<start>,MD': 0.000970873786407767,
 '<start>,FW': 0.002912621359223301,
 '<start>,NEG': 0.000970873786407767,
 '<start>,DT': 0.003883495145631068,
 '<start>,NND': 0.0048543689320388345,
 '<start>,WH': 0.000970873786407767,
 '<start>,UH': 0.002912621359223301,
 'NN,NN': 0.3253403465346535,
 'NN,SC': 0.06760519801980198,
 'NN,VB': 0.04579207920792079,
 'NN,NNP': 0.07874381188118812,
 'NN,JJ': 0.07642326732673267,
 'NN,RB': 0.017636138613861384,
 'NN,IN': 0.08245668316831684,
 'NN,Z': 0.07797029702970297,
 'NN,CD': 0.0510519801980198,
 'NN,CC': 0.0368193069306930

In [101]:
def create_emission_prob_table(word_tag, tag_count):
    emission_prob = {}
    for word_tag_entry in word_tag.keys():
        word_tag_split = word_tag_entry.split(',')
        current_word = word_tag_split[0]
        current_tag = word_tag_split[1]
        emission_key = current_word+","+current_tag
        if len(word_tag_split)==2:
            emission_prob[emission_key] = word_tag[word_tag_entry]/tag_count[current_tag]    
    return emission_prob

In [102]:
emission_prob = create_emission_prob_table(word_tag, tag_count)

In [103]:
emission_prob

{'Kera,NN': 0.0001547029702970297,
 'untuk,SC': 0.10880829015544041,
 'amankan,VB': 0.0003506311360448808,
 'pesta olahraga,NN': 0.0001547029702970297,
 'Pemerintah,NNP': 0.000517063081695967,
 'kota,NNP': 0.0007755946225439503,
 'Delhi,NNP': 0.0012926577042399173,
 'mengerahkan,VB': 0.0003506311360448808,
 'monyet,NN': 0.0018564356435643563,
 'mengusir,VB': 0.001402524544179523,
 'monyet-monyet,NN': 0.0003094059405940594,
 'lain,JJ': 0.02695167286245353,
 'yang,SC': 0.46459412780656306,
 'berbadan,VB': 0.0007012622720897616,
 'lebih,RB': 0.1423487544483986,
 'kecil,JJ': 0.013011152416356878,
 'dari,IN': 0.11091471498011489,
 'arena,NN': 0.0003094059405940594,
 'Pesta Olahraga,NNP': 0.0002585315408479835,
 'Persemakmuran,NNP': 0.0002585315408479835,
 '.,Z': 0.36773255813953487,
 'Beberapa,CD': 0.00042158516020236085,
 'laporan,NN': 0.0023205445544554454,
 'menyebutkan,VB': 0.0035063113604488078,
 'setidaknya,RB': 0.0035587188612099642,
 '10,CD': 0.00885328836424958,
 'ditempatkan,VB': 